<a href="https://colab.research.google.com/github/mirkocasu/mirkocasu.github.io/blob/main/nb/gpt-oss-(20B)-Fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Introducing FP8 precision training for faster RL inference. [Read Blog](https://docs.unsloth.ai/new/fp8-reinforcement-learning).

Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@0add68262ab0a2e33b84524346cb27cbb2787356#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

### Unsloth

We're about to demonstrate the power of the new OpenAI GPT-OSS 20B model through a finetuning example. To use our `MXFP4` inference example, use this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/GPT_OSS_MXFP4_(20B)-Inference.ipynb) instead.

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit", # 20B model using bitsandbytes 4bit quantization
    "unsloth/gpt-oss-120b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b", # 20B model using MXFP4 format
    "unsloth/gpt-oss-120b",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = dtype, # None for auto detection
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Reasoning Effort
The `gpt-oss` models from OpenAI include a feature that allows users to adjust the model's "reasoning effort." This gives you control over the trade-off between the model's performance and its response speed (latency) which by the amount of token the model will use to think.

----

The `gpt-oss` models offer three distinct levels of reasoning effort you can choose from:

* **Low**: Optimized for tasks that need very fast responses and don't require complex, multi-step reasoning.
* **Medium**: A balance between performance and speed.
* **High**: Provides the strongest reasoning performance for tasks that require it, though this results in higher latency.

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

Changing the `reasoning_effort` to `medium` will make the model think longer. We have to increase the `max_new_tokens` to occupy the amount of the generated tokens but it will give better and more correct answer

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "medium", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

Lastly we will test it using `reasoning_effort` to `high`

In [ ]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<a name="Data"></a>
### Data Prep

The `HuggingFaceH4/Multilingual-Thinking` dataset will be utilized as our example. This dataset, available on Hugging Face, contains reasoning chain-of-thought examples derived from user questions that have been translated from English into four other languages. It is also the same dataset referenced in OpenAI's [cookbook](https://cookbook.openai.com/articles/gpt-oss/fine-tune-transfomers) for fine-tuning. The purpose of using this dataset is to enable the model to learn and develop reasoning capabilities in these four distinct languages.

In [ ]:
import json
from collections import defaultdict

SYSTEM_PROMPT = (
    "You are a classifier for a virtual assistant.\n"
    "Given the conversation context (including previous turns and their annotated states), "
    "predict the target turn's mental states.\n"
    "If the target turn is by the assistant, also predict assistant_safety and assistant_failure_mode.\n"
    "If the next speaker is user (i.e., target turn is assistant and next turn exists and is user), "
    "also forecast next_user_forecast.\n\n"
    "Return ONLY valid JSON with the exact schema requested."
)

def _labels_top3(obj):
    # obj è annotation_user_states oppure annotation_assistant_states
    if obj is None:
        return []
    # nel tuo schema: {"labels_top3":[{"state":..., "score_1to5":...}, ...], "score_scale":"1-5"}
    return obj.get("labels_top3", []) or []

def _context_line(msg):
    role = msg["role"]
    content = msg.get("content", "")

    if role == "user":
        prev = {
            "role": "user",
            "content": content,
            "user_states": _labels_top3(msg.get("annotation_user_states")),
        }
    else:
        prev = {
            "role": "assistant",
            "content": content,
            "assistant_states": _labels_top3(msg.get("annotation_assistant_states")),
            "assistant_safety": msg.get("assistant_safety"),
            "assistant_failure_mode": msg.get("assistant_failure_mode"),
        }
    return prev

def build_examples_from_raw(raw_split, max_prev_turns=5):
    # raw_split: Dataset split con righe "message-level"
    by_conv = defaultdict(list)
    for r in raw_split:
        by_conv[r["conversation_id"]].append(r)

    examples = []
    for conv_id, msgs in by_conv.items():
        msgs = sorted(msgs, key=lambda x: x["turn_id"])

        for i, target in enumerate(msgs):
            prev_msgs = msgs[max(0, i - max_prev_turns): i]
            context = [_context_line(m) for m in prev_msgs]

            # Prompt: contesto (con labels precedenti) + turno target (senza labels target)
            target_desc = {
                "role": target["role"],
                "content": target.get("content", ""),
            }

            user_prompt = json.dumps(
                {
                    "conversation_id": conv_id,
                    "target_turn_id": target["turn_id"],
                    "context_previous_turns": context,
                    "target_turn": target_desc,
                    "task": {
                        "predict_current_turn_states": True,
                        "predict_assistant_safety_and_failure_if_applicable": True,
                        "forecast_next_user_states_if_applicable": True,
                        "top_k": 3,
                        "score_scale": "1-5",
                    },
                    "output_schema": {
                        "current_turn": {
                            "role": "user|assistant",
                            "user_states": [{"state": "str", "score_1to5": "int"}],
                            "assistant_states": [{"state": "str", "score_1to5": "int"}],
                            "assistant_safety": "safe|borderline|harmful|null",
                            "assistant_failure_mode": "str|null",
                        },
                        "next_user_forecast": {
                            "available": "bool",
                            "labels_top3": [{"state": "str", "score_1to5": "int"}],
                        },
                    },
                },
                ensure_ascii=False
            )

            # Ground truth output (supervision)
            gt_current = {
                "role": target["role"],
                "user_states": _labels_top3(target.get("annotation_user_states")) if target["role"] == "user" else [],
                "assistant_states": _labels_top3(target.get("annotation_assistant_states")) if target["role"] == "assistant" else [],
                "assistant_safety": target.get("assistant_safety") if target["role"] == "assistant" else None,
                "assistant_failure_mode": target.get("assistant_failure_mode") if target["role"] == "assistant" else None,
            }

            # Forecast: solo se target è assistant e il prossimo è user
            forecast_available = False
            forecast_labels = []
            if target["role"] == "assistant" and (i + 1) < len(msgs) and msgs[i + 1]["role"] == "user":
                forecast_available = True
                forecast_labels = _labels_top3(msgs[i + 1].get("annotation_user_states"))

            gt = {
                "current_turn": gt_current,
                "next_user_forecast": {
                    "available": forecast_available,
                    "labels_top3": forecast_labels if forecast_available else [],
                },
            }

            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt},
                {"role": "assistant", "content": json.dumps(gt, ensure_ascii=False)},
            ]
            examples.append({"messages": messages})

    return examples

# Costruzione split train/validation (se presente)
train_examples = build_examples_from_raw(raw["train"], max_prev_turns=5)

from datasets import Dataset
dataset = Dataset.from_list(train_examples)

if "validation" in raw:
    valid_examples = build_examples_from_raw(raw["validation"], max_prev_turns=5)
    eval_dataset = Dataset.from_list(valid_examples)
else:
    eval_dataset = None

dataset, eval_dataset

from datasets import load_dataset

# Cambia questi path (Colab: /content/... oppure Google Drive mountato)
TRAIN_JSONL = "/content/empathy_conv_v1_cleaned_with_content_colab.jsonl"
VALID_JSONL = None  # es. "/content/valid.jsonl"

data_files = {"train": TRAIN_JSONL}
if VALID_JSONL is not None:
    data_files["validation"] = VALID_JSONL

raw = load_dataset("json", data_files=data_files)
raw

To format our dataset, we will apply our version of the GPT OSS prompt

In [ ]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,
        )
        for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

if eval_dataset is not None:
    eval_dataset = eval_dataset.map(formatting_prompts_func, batched=True)

dataset[0]["text"][:500]

Let's take a look at the dataset, and check what the 1st example shows

In [ ]:
print(dataset[0]['text'])

What is unique about GPT-OSS is that it uses OpenAI [Harmony](https://github.com/openai/harmony) format which support conversation structures, reasoning output, and tool calling.

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=eval_dataset,   # <-- aggiunta (se non None)
    args=SFTConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=1,
        max_steps=30,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes and lower loss as well!

In [ ]:
from unsloth.chat_templates import train_on_responses_only

gpt_oss_kwargs = dict(instruction_part = "<|start|>user<|message|>", response_part="<|start|>assistant<|channel|>final<|message|>")

trainer = train_on_responses_only(
    trainer,
    **gpt_oss_kwargs,
)

Let's verify masking the instruction part is done! Let's print the 100th row again.

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
from transformers import TextStreamer
import json

def make_classifier_messages(conversation_id, target_turn_id, context_previous_turns, target_turn):
    user_payload = {
        "conversation_id": conversation_id,
        "target_turn_id": target_turn_id,
        "context_previous_turns": context_previous_turns,  # lista dict come in training
        "target_turn": target_turn,                        # {"role": "...", "content": "..."}
        "task": {
            "predict_current_turn_states": True,
            "predict_assistant_safety_and_failure_if_applicable": True,
            "forecast_next_user_states_if_applicable": True,
            "top_k": 3,
            "score_scale": "1-5",
        },
        "output_schema": {
            "current_turn": {
                "role": "user|assistant",
                "user_states": [{"state": "str", "score_1to5": "int"}],
                "assistant_states": [{"state": "str", "score_1to5": "int"}],
                "assistant_safety": "safe|borderline|harmful|null",
                "assistant_failure_mode": "str|null",
            },
            "next_user_forecast": {
                "available": "bool",
                "labels_top3": [{"state": "str", "score_1to5": "int"}],
            },
        },
        "constraints": [
            "Return ONLY JSON.",
            "Use top3 lists (0..3 items).",
            "Use integer score_1to5.",
        ],
    }

    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": json.dumps(user_payload, ensure_ascii=False)},
    ]

# ESEMPIO: compila tu con i tuoi ultimi 5 turni (qui mock minimale)
context_previous_turns = [
    {"role": "user", "content": "I hate it when ...", "user_states": [{"state":"embarrassment","score_1to5":5}]},
    {"role": "assistant", "content": "Ohh! that's so awkward!", "assistant_states": [{"state":"empathy","score_1to5":3}],
     "assistant_safety": "safe", "assistant_failure_mode": "none"},
]

target_turn = {"role": "assistant", "content": "Yes! Maybe you should set a reminder next time! ha ha!"}

messages = make_classifier_messages(
    conversation_id="empathy_002295_2a38f246d42e",
    target_turn_id=4,
    context_previous_turns=context_previous_turns[-5:],
    target_turn=target_turn,
)

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
).to("cuda")

streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, max_new_tokens=256, streamer=streamer)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** Currently finetunes can only be loaded via Unsloth in the meantime - we're working on vLLM and GGUF exporting!

In [ ]:
model.save_pretrained("finetuned_model")
# model.push_to_hub("hf_username/finetuned_model", token = "hf_...") # Save to HF

To run the finetuned model, you can do the below after setting `if False` to `if True` in a new instance.

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "finetuned_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 1024,
        dtype = None,
        load_in_4bit = True,
    )

messages = [
    {"role": "system", "content": "reasoning language: French\n\nYou are a helpful assistant that can solve mathematical problems."},
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high",
).to("cuda")
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

### Saving to float16 for VLLM or mxfp4

We also support saving to `float16` or `mxfp4` directly. Select `merged_16bit` for float16. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge and push to hub in mxfp4 4bit format
if False:
    model.save_pretrained_merged("finetuned_model", tokenizer, save_method = "mxfp4")
if False: model.push_to_hub_merged("repo_id/repo_name", tokenizer, token = "hf...", save_method = "mxfp4")

# Merge and push to hub in 16bit
if False:
    model.save_pretrained_merged("finetuned_model", tokenizer, save_method = "merged_16bit")
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/gpt-oss-finetune", tokenizer, save_method = "merged_16bit", token = "")

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
